In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os

In [2]:
years = [2019,2020]
file_dirs = []
for year in years:
    for filename in os.listdir(f'../data/{year}/'):
        file_dirs.append(f'../data/{year}/{filename}')
        
countries_df = [pd.read_csv(file_dir) for file_dir in file_dirs]
df = pd.concat(countries_df, ignore_index=True)
df['Period'] = pd.to_datetime(df['Period'], format='%Y%m')
cols = [
    'Period',
    'Reporter',
    'Reporter Code',
    'Trade Value (US$)',
    'Trade Flow Code',
    'Mode of Transport Code',
    'Commodity Code',
    'Commodity',
    'Customs Proc. Code',
    'Partner',
    '2nd Partner'
]
df = df[cols]

In [3]:
tot_volume_filt = (
    (df['Trade Flow Code'].isin([2, np.nan])) &  # Export only
    (df['Mode of Transport Code'].isin([0, np.nan])) &  # all modes of transport
    (df['Customs Proc. Code'].isin(['C00', np.nan])) &  # all customs protocol
    (df['Partner'] == 'World') &
    (df['2nd Partner'].isin(['World', np.nan]))
)
df = df[tot_volume_filt]
df = df.sort_values(by=['Period', 'Reporter'])

In [4]:
period = datetime.timedelta(days=90)
pre_covid_start = datetime.datetime(2019,8,1)

pre_COVID_filt = (
    (df['Period'] >= pre_covid_start) &
    (df['Period'] <= pre_covid_start + 1 * period)
)
post_COVID_filt = (
    (df['Period'] >= pre_covid_start + 2 * period) &
    (df['Period'] <= pre_covid_start + 3 * period)
)
post_COVID_df, pre_COVID_df = df[post_COVID_filt], df[pre_COVID_filt]
post_COVID_tot_vol = post_COVID_df.groupby(['Commodity Code'])['Trade Value (US$)'].sum()
pre_COVID_tot_vol = pre_COVID_df.groupby(['Commodity Code'])['Trade Value (US$)'].sum()
# risky_commodities_code = df.groupby(['Commodity Code']).sum()[(post_COVID_tot_vol > pre_COVID_tot_vol)].index
# filt = df['Commodity Code'].isin(risky_commodities_code)
# df[filt]

In [6]:
HS2_commodities = pd.read_csv('../data/HS2 key/HS2 code.csv', index_col=0)
HS2_commodities[f'change in {period.days} days'] = post_COVID_tot_vol - pre_COVID_tot_vol
HS2_commodities[HS2_commodities[f'change in {period.days} days']<0]

,Commodity,change in 90 days
HS2 Code,,
1,LIVE ANIMALS,-1.946366e+08
3,"FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUAT...",-4.387172e+09
11,PRODUCTS OF THE MILLING INDUSTRY; MALT; STARCH...,-5.466717e+07
12,OIL SEEDS AND OLEAGINOUS FRUITS; MISCELLANEOUS...,-2.561200e+09
16,"PREPARATIONS OF MEAT, OF FISH OR OF CRUSTACEAN...",-4.822914e+08
...,...,...
94,"FURNITURE; BEDDING, MATTRESSES, MATTRESS SUPPO...",-7.554871e+09
95,"TOYS, GAMES AND SPORTS REQUISITES; PARTS AND A...",-5.633219e+09
96,MISCELLANEOUS MANUFACTURED ARTICLES,-5.287573e+08
